<img src="https://i1.sndcdn.com/avatars-000274765548-vj7h0w-t500x500.jpg" style='float:right; width:200px; margin: 0 20px;'>

<img src="https://cdn2.iconfinder.com/data/icons/popular-social-media-flat/48/Popular_Social_Media-11-512.png" style='float:right; width:200px; margin: 0 20px;'>


# Twitter V CMV topological differences --> Twitter
---

> Create feature set for CMV and Twitter 

> Do comperative analysis of features

In [ ]:
# env 
import sys
sys.path.append('/Users/shaimeital/code/thesis/conversant')
sys.path.append('/Users/shaimeital/code/thesis/cmv-stance-classification')


In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import glob
import os

import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

In [2]:
from conversant.conversation.examples.controversial_feature_extraction import *
from conversant.conversation\
    .conversation_utils import conversation_to_dataframe
from stance_classification.twitter_conversation_parser import TwitterConversationReader

from stance_classification.classifiers.maxcut_stance_classifier import MaxcutStanceClassifier

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

## Load Raw Data

In [4]:
twitter_conversations_path = r"C:\Users\ronp\Documents\stance-classification\rumors_dataset\PHEME_veracity\full-conversations"
# twitter_conversations_path = "/Users/shaimeital/code/thesis/cmv-stance-classification/data/Twitter Conversation/rumers/full-conversations"
twitter_conversation_ids = [file for file in os.listdir(twitter_conversations_path)]

In [5]:
# one example
conv_id = "498235547685756928"
conversation_path = os.path.join(twitter_conversations_path, conv_id, "tweets.jsonl")
sample = pd.read_json(conversation_path,
                     lines= True,
                     dtype={'in_reply_to_status_id_str':str, 'conversation_id_str': str, 'full_text':str, 'id_str':str})
sample

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


,created_at,id_str,full_text,display_text_range,entities,extended_entities,source,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,user_id_str,retweet_count,favorite_count,reply_count,quote_count,conversation_id_str,possibly_sensitive_editable,lang
0,2014-08-10 00:32:13+00:00,498265524397432832,@annaxsweat http://t.co/Z7Qm4iaoSR,"[0, 34]",{'user_mentions': [{'screen_name': 'annaxsweat...,"{'media': [{'id_str': '498265523889930242', 'i...","<a href=""http://twitter.com/#!/download/ipad"" ...",498235547685756928,1.539100e+09,annaxsweat,21461995,2,1,0,0,498235547685756928,1.0,und
1,2014-08-10 00:37:23+00:00,498266827676741632,@annaxsweat #StopThugCops We have to take acti...,"[0, 81]","{'hashtags': [{'text': 'StopThugCops', 'indice...",NaN,"<a href=""http://twitter.com/#!/download/ipad"" ...",498235547685756928,1.539100e+09,annaxsweat,21461995,4,2,0,0,498235547685756928,NaN,en
2,2014-08-11 05:58:51+00:00,498710112358244352,@annaxsweat 18*,"[0, 15]",{'user_mentions': [{'screen_name': 'annaxsweat...,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",498235547685756928,1.539100e+09,annaxsweat,481509183,0,0,1,0,498235547685756928,NaN,und
3,2014-08-10 17:21:03+00:00,498519407375941632,@annaxsweat: Soooo wonderfully said! Thank you...,"[0, 73]",{'user_mentions': [{'screen_name': 'annaxsweat...,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",498235547685756928,1.539100e+09,annaxsweat,2331498373,0,0,0,0,498235547685756928,NaN,en
4,2014-08-10 03:40:28+00:00,498312902928244736,"@annaxsweat well, he damn sure ain't a child","[0, 44]",{'user_mentions': [{'screen_name': 'annaxsweat...,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",498235547685756928,1.539100e+09,annaxsweat,2437571275,0,1,1,0,498235547685756928,NaN,en
5,2014-08-10 03:42:21+00:00,498313376888799232,@HighPlainsCoot fuck. Off.,"[0, 26]",{},NaN,"<a href=""http://twitter.com/download/iphone"" r...",498312902928244736,2.437571e+09,HighPlainsRover,1539100118,0,0,2,0,498235547685756928,NaN,en
6,2014-08-11 10:58:25+00:00,498785502062190592,@AlexisAStevens yeah that tweet was written be...,"[0, 101]",{},NaN,"<a href=""http://twitter.com/download/iphone"" r...",498710112358244352,4.815092e+08,AlexisAurora_,1539100118,0,0,0,0,498235547685756928,NaN,en
7,2014-08-11 15:15:42+00:00,498850250048946176,@annaxsweat after several hours someone noticed,"[0, 47]",{'user_mentions': [{'screen_name': 'annaxsweat...,NaN,"<a href=""http://www.tweetcaster.com"" rel=""nofo...",498235547685756928,1.539100e+09,annaxsweat,2556027474,0,0,0,0,498235547685756928,NaN,en
8,2014-08-09 22:33:06+00:00,498235547685756928,Black teenage boys are not men. They are child...,"[0, 128]","{'hashtags': [{'text': 'ferguson', 'indices': ...",NaN,"<a href=""http://twitter.com/#!/download/ipad"" ...",nan,NaN,NaN,1539100118,177,100,9,0,498235547685756928,NaN,en
9,2014-08-09 23:04:01+00:00,498243332204949504,@annaxsweat @NeoSoulPol Same thing when #Trayv...,"[0, 131]","{'hashtags': [{'text': 'Trayvon', 'indices': [...",NaN,"<a href=""https://about.twitter.com/products/tw...",498235547685756928,1.539100e+09,annaxsweat,587926686,1,0,0,0,498235547685756928,NaN,en


In [6]:
twitter_conversations = []
errors = 0
for conv_id in tqdm(twitter_conversation_ids[:10]):
    conversation_path = os.path.join(twitter_conversations_path, conv_id, "tweets.jsonl")
    try:
        current= pd.read_json(conversation_path,
                         lines= True,
                         dtype={'in_reply_to_status_id_str':str, 'conversation_id_str': str, 'full_text':str, 'id_str':str})
        twitter_conversations.append(current)
    except:
        errors+=1
        pass

f"total number of loaded raw conversations:{len(twitter_conversations)}" \
f"\nnumber of conversations that failed to load: {errors}"

'total number of loaded raw conversations:10\nnumber of conversations that failed to load: 0'

## Transform to Conversation object and to feature set

In [7]:
twitter_reader = TwitterConversationReader(merge_sequential=True)

In [8]:
index = 0
conv_df = twitter_conversations[index]
conv_df["in_reply_to_status_id_str"].loc[conv_df["in_reply_to_status_id_str"] == "nan"] = None
conv = twitter_reader.parse(conv_df, root_id=twitter_conversation_ids[index])
conv

c:\users\ronp\py37-dasci\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


1539100118 - 498235547685756928
├── 21461995 - 498265524397432832
├── 481509183 - 498710112358244352
│   └── 1539100118 - 498785502062190592
├── 2331498373 - 498519407375941632
├── 2437571275 - 498312902928244736
│   └── 1539100118 - 498313376888799232
│       └── 2437571275 - 498314073873055744
├── 2556027474 - 498850250048946176
└── 587926686 - 498243332204949504

In [9]:
conversations = []
bad = 0
for i, conv in enumerate(tqdm(twitter_conversations)):
    if "in_reply_to_status_id_str" not in conv:
        bad += 1
        continue

    conv_id = twitter_conversation_ids[i]
    conv["in_reply_to_status_id_str"].loc[conv["in_reply_to_status_id_str"] == "nan"] = None
    conversation = twitter_reader.parse(conv, root_id=conv_id)
    conversations.append(conversation)

f"total number of conversations:{len(conversations)}" \
f"\nnumber of bad conversations: {bad}"

'total number of conversations:10\nnumber of bad conversations: 0'

In [ ]:
# sub_convs = (Conversation(child) for conv in conversations for child in conv.root.children)
#sub_convs = filter(lambda c: c.size >= 5, sub_convs)
#features = map(get_features, sub_convs)

In [ ]:
#data_with_root = pd.concat(conversation_to_dataframe(conv) for conv in sub_convs)

In [ ]:
convs = pd.concat(conversation_to_dataframe(conv) for conv in conversations)
convs

In [ ]:
convs


## EDA

### Mean branch participation for user in CMV
remove op and show that most users in a conversation only participaten in one branch

In [ ]:
roots = convs.loc[convs.is_root == True][['author', 'full_conv_id']]
roots

In [ ]:
def conversattion_to_dataframe_filter_root(conv):
    conv = conversation_to_dataframe(conv)
    full_conv_id = conv.full_conv_id[:1].values[0]
    root_author_name = roots.loc[roots.full_conv_id == full_conv_id].author.values[0]
    return conv.loc[conv.author != root_author_name]

In [ ]:
# data = pd.concat(conversattion_to_dataframe_filter_root(conv) for conv in sub_convs)

In [ ]:
data = convs

In [ ]:
user_participation=pd.DataFrame(data.groupby(['author', 'conversation_id']).timestamp.count())\
                    .rename({'timestamp': '#participation'}, axis=1)
user_participation.head()

In [ ]:
mean_branch_participation = pd.DataFrame(user_participation.groupby('author')['#participation'].mean()).\
                            rename({'#participation': 'mean_branch_participation'}, axis=1)
mean_branch_participation.head()

In [ ]:
bins = [0,1,2,3,4,5,6,7,8,9,72]

In [ ]:
mean_branch_participation['mean_par_bin'] = pd.cut(mean_branch_participation.mean_branch_participation, bins=bins)

In [ ]:
mean_branch_participation

In [ ]:
mean_branch_participation.describe()

In [ ]:
bin_participation = mean_branch_participation.groupby('mean_par_bin').count()
bin_participation

In [ ]:
bin_participation = bin_participation.reset_index()
bin_participation

In [ ]:
bin_participation.mean_branch_participation.sum()

In [ ]:
percen = bin_participation.mean_branch_participation.apply(lambda x: x / 37264)

In [ ]:
bin_participation = bin_participation.assign(perc = percen)
bin_participation

In [ ]:
_, ax = plt.subplots(figsize=(14,6))
sns.set_style("whitegrid")
ax = sns.barplot(data=bin_participation, x = 'mean_par_bin', y= 'perc', palette="deep")
plt.ylabel('percentage of twitter users')
plt.xlabel('mean participation bin')

## Some Feature Extraction

In [ ]:
# example of one branch 
conv = data.loc[data.conversation_id == 'dq95b41']
conv

In [ ]:
def count_op_involvment(subconv:pd.DataFrame, roots:pd.DataFrame) -> float:
    """
    counts the proportion of posts from the full conversation author in the subconversation
    """
    full_conversation_id = subconv.sample(1).full_conv_id.values[0]
    op = roots.loc[roots.full_conv_id == full_conversation_id].author.values[0]
    return len(subconv.loc[subconv.author == op]) / len(subconv)

In [ ]:
count_op_involvment(conv, roots)

In [ ]:
# max branch depth 
def max_branch_depth(conv:pd.DataFrame) -> int:
    return conv.depth.max()

In [ ]:
max_branch_depth(conv)

In [ ]:
# apply two features to all of the sub-conversations 
op_involvment = data.groupby('conversation_id').apply(lambda x: count_op_involvment(x, roots))

In [ ]:
op_involvment.value_counts()

In [ ]:
op_involvment_ = pd.DataFrame(op_involvment).rename({0:'op_involvment'}, axis=1)
op_involvment_

In [ ]:
feature_set = data.merge(op_involvment_, how='left', on = 'conversation_id')

## Running MAXCUT

In [ ]:
rumers_author_stance = []
for conversation in conversations:
    reply_interactions_parser = get_reply_interactions_parser()
    interaction_graph = reply_interactions_parser.parse(conversation)
    interaction_graph.get_core_interactions(inplace=True)
    interaction_graph.set_interaction_weights(lambda x: x['replies'])
    maxcut = MaxcutStanceClassifier()
    maxcut.set_input(interaction_graph.graph)
    op = conversation.root.author
    maxcut.classify_stance(op)
    for supporter in maxcut.supporters:
        row = {'conversation_id': conversation.id,
               'op': conversation.op,
               'author': supporter,
               'stance': 'supporter'}
        rumers_author_stance.append(row)
    for opposer in maxcut.complement:
        row = {'conversation_id': conversation.id,
               'op': conversation.op,
               'author': supporter,
               'stance': 'opposer'}
        rumers_author_stance.append(row)

In [ ]:
len(rumers_author_stance)

In [ ]:
user_stance_rumors = pd.DataFrame(rumers_author_stance)
user_stance_rumors

In [ ]:
user_stance_rumors.dtypes

In [ ]:
user_stance_rumors.conversation_id.nunique()

## Gold Standard

In [ ]:
path = r"C:\Users\ronp\Documents\stance-classification\rumors_dataset\pheme-rumour-scheme-dataset\annotations\en-scheme-annotations.json"
# path = "/Users/shaimeital/code/thesis/cmv-stance-classification/Examples/en-scheme-annotations_clean.jsonl"
rumors_gold_label = pd.read_json(path, lines = True)
rumors_gold_label


In [ ]:
rumors_gold_label['responsetype-vs-source'].value_counts()

In [ ]:
rumors_gold_label = rumors_gold_label.filter(['threadid', 'tweetid', 'responsetype-vs-source'], axis=1).rename({'responsetype-vs-source': 'label'}, axis=1)
rumors_gold_label

In [ ]:
rumors_gold_label.threadid = rumors_gold_label.threadid.astype('str') 

In [ ]:
rumors_gold_label.tweetid = rumors_gold_label.tweetid.astype('str') 

In [ ]:
rumors_gold_label.dtypes

In [ ]:
convs_a = convs_.filter(['threadid', 'tweetid', 'author'], axis=1)
convs_a

In [ ]:
convs_a.dtypes

In [ ]:
rumors_gold_label_ = rumors_gold_label.merge(convs_a, how = 'left', on=['threadid', 'tweetid'])
rumors_gold_label_

In [ ]:
rumors_gold_label_.author.isna().sum()

In [ ]:
rumors_gold_label_nona = rumors_gold_label_.dropna()
rumors_gold_label_nona

In [ ]:
user_stance_rumors = user_stance_rumors.rename({'conversation_id': 'threadid', 'stance':'stance_prediction'}, axis=1)
user_stance_rumors

In [ ]:
predictions = user_stance_rumors.merge(rumors_gold_label_nona, how='left', on = ['threadid', 'author'])
predictions

In [ ]:
predictions_ = predictions.dropna()
predictions_

In [ ]:
predictions_.loc[(predictions_.stance_prediction == 'supporter') & (predictions_.label == 'agreed') ]

In [ ]:
predictions_.loc[(predictions_.stance_prediction == 'opposer') & (predictions_.label == 'disagreed') ]

In [ ]:
predictions_.loc[(predictions_.stance_prediction == 'supporter') & (predictions_.label == 'disagreed') ]

In [ ]:
predictions_.loc[(predictions_.stance_prediction == 'opposer') & (predictions_.label == 'agreed') ]

## Extra for TSNE + PCA

In [ ]:
# data = pd.DataFrame.from_records(features, index="conv_id")
# print(data.head())

In [ ]:
data_ = data.loc[data.num_nodes > 10]

In [ ]:
data_.count()

In [ ]:
data_

In [ ]:
data_.describe()

In [ ]:
data_ = data_.fillna(0)

In [ ]:
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler


In [ ]:
standardized_data = StandardScaler().fit_transform(data_)
print(standardized_data.shape)

In [ ]:
model = TSNE(n_components=2, random_state=0)

In [ ]:
tsne_data = model.fit_transform(standardized_data)

In [ ]:
tsne_df = pd.DataFrame(data=tsne_data, columns=("Dim_1", "Dim_2"))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.FacetGrid(tsne_df, height=9).map(plt.scatter, 'Dim_1', 'Dim_2').add_legend()
plt.show()

In [ ]:
from sklearn.cluster import DBSCAN


In [ ]:
m = DBSCAN()


In [ ]:
m.fit(standardized_data)

In [ ]:
clusters = m.labels_

In [ ]:
pd.Series(clusters).value_counts()

In [ ]:
# pca

from sklearn.decomposition import PCA


In [ ]:
pca = PCA(n_components=2)


In [ ]:
principalComponents = pca.fit_transform(standardized_data)

In [ ]:
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

In [ ]:
sns.FacetGrid(principalDf, height=6).map(plt.scatter, 'principal component 1', 'principal component 2').add_legend()
plt.show()